In [354]:
import pandas as pd
import numpy as np
import seaborn as sns
import pickle
import matplotlib.pyplot as plt
from IPython.display import display
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.feature_selection import SelectFromModel, RFE
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
import statistics
#import missingno as msno

In [355]:
path_file = 'R1400.xlsx'
sheets_excel = pd.read_excel('R1400.xlsx',sheet_name=None)

In [356]:
for name in list(sheets_excel.keys()):
    sheets_excel[name].to_csv(name+'.csv', index=False)

### P4_S01

In [398]:
DF_P4_S1 = pd.read_csv('R1400P4S01.csv')

In [399]:
DF_P4_S1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10731 entries, 0 to 10730
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Address        10731 non-null  int64  
 1   member         10731 non-null  int64  
 2   employed_w     10731 non-null  int64  
 3   ISCO_w         10731 non-null  int64  
 4   ISIC_w         10731 non-null  int64  
 5   status_w       10731 non-null  int64  
 6   hours_w        9786 non-null   float64
 7   days_w         9787 non-null   float64
 8   income_w_m     10731 non-null  int64  
 9   income_w_y     10731 non-null  int64  
 10  wage_w_m       10718 non-null  float64
 11  wage_w_y       10726 non-null  float64
 12  perk_w_m       10721 non-null  float64
 13  perk_w_y       10726 non-null  float64
 14  netincome_w_m  10731 non-null  int64  
 15  netincome_w_y  10731 non-null  int64  
 16  Fasl           10731 non-null  int64  
 17  year           10731 non-null  int64  
 18  DYCOL0

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=5 color= orange>
ستون DYcol00 کامل nan است.

In [400]:
DF_P4_S1.drop(['DYCOL00'], axis=1, inplace=True)

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=5 color= orange>
همبستگی ستون‌های income_w , wage_w, netincome_w مورد برسی قرار گرفت و چون همستگی ستون income در سال‌های دیگه کم بود فقط ستون wage_w حذف میشود.

In [401]:
DF_P4_S1[['income_w_m', 'wage_w_m', 'netincome_w_m']].corr()

,income_w_m,wage_w_m,netincome_w_m
income_w_m,1.000000,0.863049,0.890568
wage_w_m,0.863049,1.000000,0.966855
netincome_w_m,0.890568,0.966855,1.000000


In [402]:
DF_P4_S1[['income_w_y', 'wage_w_y', 'netincome_w_y']].corr()

,income_w_y,wage_w_y,netincome_w_y
income_w_y,1.000000,0.305375,0.309269
wage_w_y,0.305375,1.000000,0.987100
netincome_w_y,0.309269,0.987100,1.000000


In [403]:
DF_P4_S1.drop(['wage_w_m'], axis=1 , inplace= True)
DF_P4_S1.drop(['wage_w_y'], axis=1 , inplace= True)

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=5 color= orange>
با توجه به اطلاعات زیر ستون‌های employed_w, status_w, perk_w_y, perk_w_m دارای imblance هستند بنابراین حذف می‌کنیم.

In [404]:
for col in DF_P4_S1.columns:
 print(f'{col :>15} {(DF_P4_S1[col].value_counts()/len(DF_P4_S1)).max():f}')

        Address 0.000652
         member 0.690336
     employed_w 0.893020
         ISCO_w 0.308732
         ISIC_w 0.297083
       status_w 0.901500
        hours_w 0.539372
         days_w 0.374709
     income_w_m 0.133538
     income_w_y 0.045196
       perk_w_m 0.965334
       perk_w_y 0.781940
  netincome_w_m 0.133725
  netincome_w_y 0.045569
           Fasl 0.266797
           year 1.000000


In [405]:
DF_P4_S1.drop(['employed_w', 'status_w', 'perk_w_y', 'perk_w_m'], inplace=True, axis=1)

In [406]:
DF_P4_S1.isna().sum()

Address            0
member             0
ISCO_w             0
ISIC_w             0
hours_w          945
days_w           944
income_w_m         0
income_w_y         0
netincome_w_m      0
netincome_w_y      0
Fasl               0
year               0
dtype: int64

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=5 color= orange>
ستون‌های day_w , hours_w با گروه بندی روی ISCO_w, ISIC_w پر می‌کنیم و ما بقی را با میانه کل داده‌ها پر می‌کنیم.

In [407]:
temp = DF_P4_S1.groupby(['ISCO_w', 'ISIC_w' ])[['hours_w','days_w']].transform('median')
#temp = np.floor(temp).astype(pd.Float64Dtype())
DF_P4_S1.loc[DF_P4_S1.hours_w.isna(), 'hours_w'] = temp.loc[DF_P4_S1.hours_w.isna(), 'hours_w']
DF_P4_S1.loc[DF_P4_S1.days_w.isna(), 'days_w'] = temp.loc[DF_P4_S1.days_w.isna(), 'days_w']

In [408]:
DF_P4_S1.isna().sum()

Address           0
member            0
ISCO_w            0
ISIC_w            0
hours_w          47
days_w           47
income_w_m        0
income_w_y        0
netincome_w_m     0
netincome_w_y     0
Fasl              0
year              0
dtype: int64

In [409]:
DF_P4_S1.hours_w.fillna(DF_P4_S1.hours_w.median(),inplace= True)
DF_P4_S1.days_w.fillna(DF_P4_S1.days_w.median(), inplace= True)

In [410]:
DF_P4_S1.isna().sum()

Address          0
member           0
ISCO_w           0
ISIC_w           0
hours_w          0
days_w           0
income_w_m       0
income_w_y       0
netincome_w_m    0
netincome_w_y    0
Fasl             0
year             0
dtype: int64

In [411]:
path_file = 'R1400_P4_S01'
DF_P4_S1.to_csv(path_file+'.csv', index=False)

### P4_S02

In [412]:
DF_P4_S2 = pd.read_csv('R1400P4S02.csv')
DF_P4_S2.head()

,Address,member,employed_s,ISCO_s,ISIC_s,status_s,agriculture,hours_s,days_s,cost_employment,cost_raw,cost_machinery,cost_others,cost_tax,sale,income_s_y,Fasl,year,DYCOL00
0,20001384232,1,1,6111,1110,5,1,9.0,7.0,9000000.0,85000000.0,NaN,79000000.0,NaN,970000000.0,797000000,1,1401,NaN
1,20011394623,1,1,6111,1110,5,1,10.0,3.0,NaN,53500000.0,NaN,78000000.0,NaN,205000000.0,73500000,1,1401,NaN
2,20011394623,1,1,6121,1440,5,1,3.0,7.0,NaN,183000000.0,NaN,4000000.0,NaN,360000000.0,173000000,1,1401,NaN
3,20011394623,3,1,6121,1440,6,1,10.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,0,1,1401,NaN
4,20003387029,1,1,6112,1240,4,1,5.0,4.0,50000000.0,96000000.0,NaN,NaN,NaN,245000000.0,99000000,1,1401,NaN


In [413]:
DF_P4_S2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12194 entries, 0 to 12193
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Address          12194 non-null  int64  
 1   member           12194 non-null  int64  
 2   employed_s       12194 non-null  int64  
 3   ISCO_s           12194 non-null  int64  
 4   ISIC_s           12194 non-null  int64  
 5   status_s         12194 non-null  int64  
 6   agriculture      12194 non-null  int64  
 7   hours_s          12058 non-null  float64
 8   days_s           12055 non-null  float64
 9   cost_employment  10807 non-null  float64
 10  cost_raw         11887 non-null  float64
 11  cost_machinery   10908 non-null  float64
 12  cost_others      11496 non-null  float64
 13  cost_tax         10463 non-null  float64
 14  sale             11989 non-null  float64
 15  income_s_y       12194 non-null  int64  
 16  Fasl             12194 non-null  int64  
 17  year        

In [414]:
DF_P4_S2.drop(['DYCOL00'], axis=1 , inplace= True)

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=5 color= orange>
ستون‌های cost جمع می‌شوند و در ستون جدیدی به نام Total_cost ذخیره می‌شوند.

In [415]:
del_columns = ['cost_employment', 'cost_raw', 'cost_machinery', 'cost_others', 'cost_tax']
DF_P4_S2.insert(9,'Total_cost', DF_P4_S2.loc[:,'cost_employment':'cost_tax' ].sum(axis=1))
DF_P4_S2.drop(del_columns, axis=1, inplace=True)

In [375]:
DF_P4_S2.isna().sum()

Address          0
member           0
employed_s       0
ISCO_s           0
ISIC_s           0
status_s         0
agriculture      0
hours_s        136
days_s         139
Total_cost       0
sale           205
income_s_y       0
Fasl             0
year             0
dtype: int64

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=5 color= orange>
ستون‌های day_s , hours_s با گروه بندی روی ISCO_s, ISIC_s پر می‌کنیم و ما بقی را با میانه کل داده‌ها پر می‌کنیم.

In [416]:
temp = DF_P4_S2.groupby(['ISCO_s', 'ISIC_s' ])[['hours_s','days_s']].transform('median')
#temp = np.floor(temp).astype(pd.Int16Dtype())
DF_P4_S2.loc[DF_P4_S2.hours_s.isna(), 'hours_s'] = temp.loc[DF_P4_S2.hours_s.isna(), 'hours_s']
DF_P4_S2.loc[DF_P4_S2.days_s.isna(), 'days_s'] = temp.loc[DF_P4_S2.days_s.isna(), 'days_s']

DF_P4_S2.hours_s.fillna(DF_P4_S2.hours_s.median(),inplace= True)
DF_P4_S2.days_s.fillna(DF_P4_S2.days_s.median(), inplace= True)

In [417]:
DF_P4_S2.isna().sum()

Address          0
member           0
employed_s       0
ISCO_s           0
ISIC_s           0
status_s         0
agriculture      0
hours_s          0
days_s           0
Total_cost       0
sale           205
income_s_y       0
Fasl             0
year             0
dtype: int64

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=5 color= orange>
ستون sale
همان طور که در کد زیر نشان داده شده است تقریبن در 90 درصد موارد nan در ستون income_s_y مقدار0 دارد.

In [418]:
DF_P4_S2.loc[DF_P4_S2.sale.isna()].income_s_y.value_counts() / DF_P4_S2.sale.isna().sum() * 100

income_s_y
0             88.780488
440000000      0.975610
340000000      0.975610
720000000      0.975610
600000000      0.487805
790000000      0.487805
890000000      0.487805
1250000000     0.487805
740000000      0.487805
650000000      0.487805
660000000      0.487805
1350000000     0.487805
680000000      0.487805
270000000      0.487805
100000000      0.487805
430000000      0.487805
530000000      0.487805
560000000      0.487805
380000000      0.487805
420000000      0.487805
580000000      0.487805
Name: count, dtype: float64

In [419]:
DF_P4_S2.loc[DF_P4_S2.sale.isna()].status_s.value_counts() 

status_s
6    180
5     25
Name: count, dtype: int64

In [420]:
DF_P4_S2.fillna(0, inplace=True)

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=5 color= orange>
ستون employed_s به دلیل imblance حذف می‌کنیم.

In [421]:
for col in DF_P4_S2.columns:
 print(f'{col :>15} {(DF_P4_S2[col].value_counts()/len(DF_P4_S2)).max():f}')

        Address 0.000656
         member 0.721010
     employed_s 0.985813
         ISCO_s 0.340085
         ISIC_s 0.251107
       status_s 0.724127
    agriculture 0.746433
        hours_s 0.190750
         days_s 0.438740
     Total_cost 0.226915
           sale 0.200426
     income_s_y 0.189601
           Fasl 0.255618
           year 1.000000


In [422]:
DF_P4_S2.drop(['employed_s'],axis=1, inplace=True)

In [423]:
DF_P4_S2.isna().sum()

Address        0
member         0
ISCO_s         0
ISIC_s         0
status_s       0
agriculture    0
hours_s        0
days_s         0
Total_cost     0
sale           0
income_s_y     0
Fasl           0
year           0
dtype: int64

In [424]:
path_file = 'R1400_P4_S02'
DF_P4_S2.to_csv(path_file+'.csv', index=False)

### P4_S03

In [425]:
DF_P4_S3 = pd.read_csv('R1400P4S03.csv')
DF_P4_S3.head()

,Address,member,income_pension,income_rent,income_interest,income_aid,income_resale,income_transfer,Fasl,year,DYCOL00
0,20001384225,1,NaN,NaN,NaN,25600000.0,NaN,NaN,1,1401,NaN
1,20001384232,1,NaN,NaN,NaN,25600000.0,NaN,NaN,1,1401,NaN
2,20001384235,1,NaN,0300000000,240000000.0,20800000.0,NaN,NaN,1,1401,NaN
3,20011394620,1,NaN,NaN,20640000.0,20000000.0,NaN,NaN,1,1401,NaN
4,20011394623,1,NaN,NaN,17000000.0,10000000.0,NaN,NaN,1,1401,NaN


In [426]:
DF_P4_S3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25097 entries, 0 to 25096
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Address          25097 non-null  int64  
 1   member           25097 non-null  int64  
 2   income_pension   7387 non-null   object 
 3   income_rent      6002 non-null   object 
 4   income_interest  18122 non-null  float64
 5   income_aid       18249 non-null  float64
 6   income_resale    5579 non-null   float64
 7   income_transfer  8478 non-null   float64
 8   Fasl             25097 non-null  int64  
 9   year             25097 non-null  int64  
 10  DYCOL00          0 non-null      float64
dtypes: float64(5), int64(4), object(2)
memory usage: 2.1+ MB


In [427]:
DF_P4_S3.drop(['DYCOL00'], axis = 1, inplace=True)

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=5 color= orange>
ستون‌های income_pension , income_rent به float تغییر می‌دهیم.

In [428]:
DF_P4_S3.income_pension = DF_P4_S3.income_pension.str.strip()
DF_P4_S3.loc[DF_P4_S3.income_pension.isin(['']) , 'income_pension' ] = np.nan
DF_P4_S3.income_pension = DF_P4_S3.income_pension.astype(pd.Float64Dtype())

DF_P4_S3.income_rent = DF_P4_S3.income_rent.str.strip()
DF_P4_S3.loc[DF_P4_S3.income_rent.isin(['']) , 'income_rent' ] = np.nan
DF_P4_S3.income_rent = DF_P4_S3.income_rent.astype(pd.Float64Dtype())


<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=5 color= orange>
ستون‌های income جمع می‌شوند و در ستون جدیدی به نام Total_income ذخیره می‌شوند.

In [429]:
DF_P4_S3.insert(2,'Total_income', DF_P4_S3.loc[:,'income_pension':'income_transfer' ].sum(axis=1))
del_columns = ['income_pension','income_rent', 'income_interest','income_aid','income_resale','income_transfer']
DF_P4_S3.drop(del_columns,axis=1, inplace=True)

In [390]:
DF_P4_S3.isna().sum()

Address         0
member          0
Total_income    0
Fasl            0
year            0
dtype: int64

In [391]:
path_file = 'R1400_P4_S03'
DF_P4_S3.to_csv(path_file+'.csv', index=False)

### P4_S04

In [392]:
DF_P4_S4 = pd.read_csv('R1400P4S04.csv')
DF_P4_S4.head()

,Address,member,subsidy_number,subsidy_month,subsidy,Fasl,year,DYCOL00
0,20001384225,1,1.0,12.0,1960000,1,1401,NaN
1,20001384225,2,2.0,12.0,10920000,1,1401,NaN
2,20001384232,1,1.0,12.0,1960000,1,1401,NaN
3,20001384232,2,2.0,12.0,10920000,1,1401,NaN
4,20001384235,1,1.0,12.0,1960000,1,1401,NaN


In [393]:
DF_P4_S4.drop(['DYCOL00'], axis = 1, inplace=True)

In [394]:
DF_P4_S4.isna().sum()

Address           0
member            0
subsidy_number    4
subsidy_month     1
subsidy           0
Fasl              0
year              0
dtype: int64

In [395]:
DF_P4_S4.dropna(inplace=True)

In [396]:
DF_P4_S4.isna().sum()

Address           0
member            0
subsidy_number    0
subsidy_month     0
subsidy           0
Fasl              0
year              0
dtype: int64

In [397]:
path_file = 'R1400_P4_S04'
DF_P4_S4.to_csv(path_file+'.csv', index=False)